In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
indep=dataset[["Gender_Male","Age",'EstimatedSalary']]
dep=dataset[["Purchased"]]

In [7]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid={'criterion':["gini", "entropy", "log_loss"], 'splitter': ["best", "random"], 'class_weight': ['balanced', 'dict', None], 'max_features': [None,"sqrt", "log2"]}
grid= GridSearchCV(DecisionTreeClassifier(), param_grid, refit= True, verbose=3, n_jobs=-1, scoring= 'f1_weighted' )
grid.fit(X_train,y_train)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


C:\Anaconda3run\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
90 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Anaconda3run\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Anaconda3run\Lib\site-packages\sklearn\base.py", line 1144, in wrapper
    estimator._validate_params()
  File "C:\Anaconda3run\Lib\site-packages\sklearn\base.py", line 637, in _validate_params
    validate_parameter_constraints(
  File "C:\Anaconda3run\Lib\site-packages\sklearn\utils\_param_validation.py", line 95

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'dict', None],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': [None, 'sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [11]:
y_pred=grid.predict(X_test)

In [12]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
cm

array([[76,  9],
       [ 8, 41]], dtype=int64)

In [13]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,y_pred)

In [14]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.89      0.90        85
           1       0.82      0.84      0.83        49

    accuracy                           0.87       134
   macro avg       0.86      0.87      0.86       134
weighted avg       0.87      0.87      0.87       134



In [15]:
from sklearn.metrics import roc_auc_score
roc_score= roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])
roc_score

0.8654261704681873

In [16]:
print ( " the best parameter is ", grid.best_params_)

 the best parameter is  {'class_weight': None, 'criterion': 'entropy', 'max_features': None, 'splitter': 'best'}


In [17]:
grid.cv_results_

{'mean_fit_time': array([0.00657864, 0.00517993, 0.00658264, 0.00518355, 0.0057795 ,
        0.00698047, 0.00617657, 0.0035881 , 0.00438867, 0.00458755,
        0.00458851, 0.00479188, 0.00398989, 0.00418944, 0.00399027,
        0.00379176, 0.00398598, 0.00418921, 0.00100021, 0.00059819,
        0.00079846, 0.00079799, 0.00079856, 0.00039887, 0.00080366,
        0.00059876, 0.00039907, 0.00079799, 0.00079923, 0.0009995 ,
        0.00099778, 0.00079794, 0.00119915, 0.00099506, 0.00100088,
        0.00120015, 0.00418892, 0.00378928, 0.00299039, 0.00339212,
        0.0031919 , 0.00359025, 0.00378633, 0.00339174, 0.00378957,
        0.00418892, 0.00318666, 0.00359063, 0.00358791, 0.00319114,
        0.00439215, 0.00538979, 0.00438805, 0.00319147]),
 'std_fit_time': array([1.19833762e-03, 7.40357731e-04, 1.73939697e-03, 1.16971454e-03,
        9.75703734e-04, 2.81770263e-03, 7.45108215e-04, 4.87288909e-04,
        4.89220112e-04, 1.35237434e-03, 7.97879858e-04, 1.16386820e-03,
        4.623

In [18]:
re=grid.cv_results_

In [19]:
table= pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006579,1.198338e-03,0.014960,0.003568,balanced,gini,None,best,"{'class_weight': 'balanced', 'criterion': 'gin...",0.826263,0.868752,0.833323,0.906935,0.924528,0.871960,0.038931,2
1,0.005180,7.403577e-04,0.012567,0.001850,balanced,gini,None,random,"{'class_weight': 'balanced', 'criterion': 'gin...",0.808927,0.811321,0.775815,0.795256,0.829124,0.804089,0.017776,33
2,0.006583,1.739397e-03,0.020150,0.004568,balanced,gini,sqrt,best,"{'class_weight': 'balanced', 'criterion': 'gin...",0.847141,0.846714,0.814409,0.832483,0.847020,0.837553,0.012859,22
3,0.005184,1.169715e-03,0.014756,0.001940,balanced,gini,sqrt,random,"{'class_weight': 'balanced', 'criterion': 'gin...",0.799620,0.811321,0.813179,0.849057,0.826499,0.819935,0.016874,29
4,0.005780,9.757037e-04,0.016759,0.003116,balanced,gini,log2,best,"{'class_weight': 'balanced', 'criterion': 'gin...",0.826263,0.870047,0.814409,0.831098,0.883278,0.845019,0.026731,15
5,0.006980,2.817703e-03,0.015163,0.001715,balanced,gini,log2,random,"{'class_weight': 'balanced', 'criterion': 'gin...",0.743022,0.850809,0.740173,0.832483,0.829124,0.799122,0.047554,35
6,0.006177,7.451082e-04,0.015359,0.002570,balanced,entropy,None,best,"{'class_weight': 'balanced', 'criterion': 'ent...",0.832255,0.849057,0.832483,0.870362,0.905069,0.857845,0.027430,8
7,0.003588,4.872889e-04,0.010970,0.000001,balanced,entropy,None,random,"{'class_weight': 'balanced', 'criterion': 'ent...",0.885356,0.815954,0.832483,0.851527,0.923510,0.861766,0.038549,6
8,0.004389,4.892201e-04,0.010572,0.001196,balanced,entropy,sqrt,best,"{'class_weight': 'balanced', 'criterion': 'ent...",0.826263,0.831253,0.850543,0.870362,0.841025,0.843889,0.015644,16
9,0.004588,1.352374e-03,0.011768,0.003115,balanced,entropy,sqrt,random,"{'class_weight': 'balanced', 'criterion': 'ent...",0.791752,0.790949,0.793565,0.814409,0.811321,0.800399,0.010260,34


In [20]:
grid.best_score_

0.8801657906849198

In [21]:
pre_input= sc.transform([[35,60000,0]])
pre_input

C:\Anaconda3run\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[ 6.90094695e+01,  5.91413422e+03, -2.05850391e+00]])

In [22]:
result= grid.predict(pre_input)
result

array([1], dtype=int64)